In [1]:
import sys, time, random, math, cv2, os
import numpy as np

sys.path.insert(0, '../src')
from robot import Robot

In [2]:
N_STATE_AO = 2**4
actions_ao = np.array([(3,3), (1,-1), (-1,1)])

dimensions = (N_STATE_AO, len(actions_ao))

path = "Q_matrix_ao.txt"
on_disk = path in os.listdir()

if on_disk:
    f = open(path, "r")
    
    t = f.read().split()
    iterations = int(t[0])
    random_rate = float(t[1])
    learning_rate = float(t[2])
    
    Q = np.reshape(np.array([float(i) for i in t[3:]]), dimensions)

    f.close()
else:
    iterations = 0
    random_rate = 0.9
    learning_rate = 0.6
    Q = np.zeros(dimensions)

In [3]:
def get_state_avoid_obstacle(us_distances):
    threshold = 0.7
    r = 0
    r += 1 * us_distances[2] <= threshold
    r += 2 * us_distances[3] <= threshold
    r += 4 * us_distances[4] <= threshold
    r += 8 * us_distances[5] <= threshold
    
    return r

def get_action_avoid_obstacle(Q, state, random_rate):
    a = random.random()
    if a < random_rate:
        i = random.randint(0, len(actions_gtg) - 1)
    else:
        i = 0
        for j in range(len(actions_gtg)):
            if Q[state][j] > Q[state][i]:
                i = j
    return i

def got_reward_avoid_obstacle(curve, us_distances):
    threshold = 0.1
    flag = False
    for i in us_distances[2:6]:
        flag = flag or i <= threshold
    if flag:
        return -1000
    
    if curve:
        return 0
    
    return 100

In [ ]:
obot = Robot("#0")

max_iterations = 1000

while robot.get_connection_status() != -1 and iterations < max_iterations:
    us_distances = robot.read_ultrassonic_sensors()
    
    state = get_state_avoid_obstacle(us_distances)
    
    i = get_action_avoid_obstacle(Q, state, random_rate)
    
    robot.set_left_velocity(actions_ao[i][0])
    robot.set_right_velocity(actions_ao[i][1])
    time.sleep(3)
    robot.set_left_velocity(0)
    robot.set_right_velocity(0)

    reward = get_reward_avoid_obstacle(actions_ao[i][0] != actions_ao[i][1])
    
    print("state: " + str(state) + " action: " + str(ac) + " reward: " + str(reward))
    
    best_q = np.amax(Q[state])

    Q[state][ac] = learning_rate * (reward + 0.99 * best_q - Q[state][ac])
    
    if get_distance_from_goal(new_position, goal[i]):
        i = (i + 1) % 2
    
    random_rate = max(0.1, random_rate * 0.9995)
    learning_rate = max(0.1, learning_rate * 0.9999)

    f = open(path, "w")
    f.write(str(iterations) + "\n")
    f.write(str(random_rate) + "\n")
    f.write(str(learning_rate) + "\n")

    for q in Q:
        for j in q:
            f.write(str(j) + " ")
        f.write("\n")
    f.close()

    iterations += 1